# [Broken Clock](https://codingcompetitions.withgoogle.com/codejam/round/0000000000435baf/00000000007ae694)
- Given: A clock with faded numbers, broken hands
    + The clock is divided into `12 * 60 * 60 * 10^9` ticks
    + 3 broken hands: hour, minute, second
    + `A,B,C`: the position of 3 broken hands (can't distinguish which is which, only know as A,B,C) at the moment versus 1 imaginary axis in `tick` units
        + hour hand: 1 nano second moves 1 tick (maximum 12 hrs)
        + minute hand: 1 nano second moves 12 tick (maximum 60 mins)
        + second hand: 1 nano second moves 720 tick (maximum 60 seconds)

- Find the correct time in `h:m:s:n` format


#### Constraints
- $0 \leq A,B,C \leq 12 * 60 * 60 * 10^9$
- Test set 1:
    + Knowing that the clock is not rotated -> knowing the imaginary axis is at 12am
    + `A,B,C` is given in second precision (ans = `h:m:s:0`)
- Test set 2:
    + The clock is rotated -> the imaginary axis can be at anytime
    + `A,B,C` is given in second precision (ans = `h:m:s:0`)
- Test set 3:
    + The clock is rotated -> the imaginary axis can be at anytime
    + `A,B,C` is given in nano second precision (ans = `h:m:s:n`)
    
#### Examples
- Input

```
<!--  Testset 1 -->
0 0 0
0 21600000000000 23400000000000
1476000000000 2160000000000 3723000000000

<!--  Testset 2 -->
5400000000000 5400000000000 5400000000000
10800000000000 32400000000000 34200000000000
23076000000000 23760000000000 25323000000000

<!--  Testset 3 -->
0 11 719
```

- Output

```
Case #1: 0 0 0 0
Case #2: 6 30 0 0
Case #3: 1 2 3 0
Case #4: 0 0 0 0
Case #5: 6 30 0 0
Case #6: 1 2 3 0
Case #7: 0 0 0 1
```

# Solution
- System of equations
    + Assume: A = hour hand, B = minute hand, C = second hand
    + Let x is the answer, correct time in `ticks` units
    + offset is the relative position of the imaginary axis vs 12am (= 0 in testset 1)
    
$$\begin{cases}
    x + \text{offset} = A \\
    12x + \text{offset} = B \\
    720x + \text{offset} = C \\
\end{cases}$$

- Simplify then we can solve for x while knowing A,B,C

$$\begin{cases}
    x = \frac{B-A}{11} \\
    719x = C - A
\end{cases}$$

- permute A,B,C to search for the correct x.
    + x with the maximum value = correct time ticks
    + Convert ticks to `h:m:s:n` by `ticks_2_hmsn()`
    
#### Notes
- Consider integer overflown while doing modulus operations
- For this problem, Use `int64_t` for sub, add, and `__int128` when multiply 2 `int64_t` nums

#### Codes

```C++
/* ------------------ MODULUS API-----------------------------*/
const int64_t MOD = 12 * 60 * 60 * (int64_t)1e9;

inline int64_t sub(int64_t a,int64_t b) {
    if(a<0) a = (a%MOD) + MOD;
    if(b<0) b = (b%MOD) + MOD;
    return ((a%MOD) - (b%MOD) + MOD) % MOD;
}
inline int64_t mul(int64_t a,int64_t b) {
    if(a<0) a = (a%MOD) + MOD;
    if(b<0) b = (b%MOD) + MOD;
    return ((__int128)(a%MOD) * (b%MOD)) % MOD;
}

inline int64_t gcd(int64_t a, int64_t b) {
    assert(a >= 0 and b >= 0 and a + b > 0);

    while(a>0 and b>0) {
        if(a >= b) a %= b;
        else b %= a;
    }
    return max(a,b);
}
inline vector<int64_t> extendedGCD(int64_t a, int64_t b) {
    assert(a >= b and b >= 0 and a + b > 0);

    if(b == 0) return {a,1,0};

    vector<int64_t> dpq = extendedGCD(b, a%b);
    int64_t x = dpq[2];
    int64_t y = dpq[1] - dpq[2] * (a/b);

    assert(a % dpq[0] == 0 and b % dpq[0] == 0);
    assert(dpq[0] == a * x + b * y);
    return {dpq[0],x,y};
}
inline int64_t inv(int64_t b) {
    if(b<0) b = (b%MOD) + MOD;
    assert(MOD > 1 and b > 0 and gcd(b, MOD) == 1);
    vector<int64_t> dts = extendedGCD(MOD, b%MOD);
    return dts[2];
}
inline int64_t divi(int64_t a, int64_t b) {
    if(a<0) a = (a%MOD) + MOD;
    if(b<0) b = (b%MOD) + MOD;
    return mul(a,inv(b));
}


/* ------------------ Solve -----------------------------*/
int A,B,C;


int get(int A, int B, int C) {
    int x = divi(sub(B,A), 11);
    if(mul(719, x) != sub(C,A)) return -1;
    return x;
}

vector<int> ticks_2_hmsn(int ticks) {
    int tmp = ticks;

    int n = tmp % (int)1e9;
    tmp /= (int)1e9;

    int s = tmp % 60;
    tmp /= 60;

    int m = tmp % 60;
    tmp /= 60;

    int h = tmp % 12;
    return vector<int>({h,m,s,n});
}

vector<int> sol() {
    vector<int> hands({A,B,C});
    int ticks = 0;
    do {
        ticks = max(ticks, get(hands[0], hands[1], hands[2]));
    } while(next_permutation(hands.begin(), hands.end()));
    return ticks_2_hmsn(ticks);
} 
```